In [ ]:
from spectralmc.sobol_sampler import SobolSampler, BoundSpec
from spectralmc.gbm import SimulationParams, BlackScholes
import pandas as pd
from typing import Optional

In [ ]:
sp = SimulationParams(
    timesteps=1024,
    network_size=256,
    batches_per_mc_run=1024,
    threads_per_block=256,
    mc_seed=42,
    buffer_size=4,
)

In [ ]:
total_size_mb=sp.memory_footprint_bytes()/1024/1024
print(total_size_mb)

In [ ]:
bs_dimensions = {
    "X0": BoundSpec(lower=0.001, upper=10000),
    "K":  BoundSpec(lower=0.001, upper=20000),
    "T":  BoundSpec(lower=0,     upper=10),
    "r":  BoundSpec(lower=-0.2,  upper=0.2),
    "d":  BoundSpec(lower=-0.2,  upper=0.2),
    "v":  BoundSpec(lower=0,     upper=2.0),
}

In [ ]:
ss = SobolSampler(pydantic_class=BlackScholes.Inputs,dimensions=bs_dimensions,seed=43)

In [ ]:
bs = BlackScholes(sp=sp)

In [ ]:
%%time
samples=ss.sample(128)

In [ ]:
%%time
prices=[bs.price_to_host(input) for input in samples]

In [ ]:
bs._normal_gen.get_time_spent_synchronizing()

In [ ]:
from __future__ import annotations

import math
from typing import Final

import QuantLib as ql  # type: ignore[import]

# assume BlackScholes (with Inputs and HostPricingResults) is already defined


def bs_price_quantlib(
    inp: BlackScholes.Inputs,
) -> BlackScholes.HostPricingResults:
    """
    Analytic Black-Scholes–Merton pricer using *QuantLib*’s
    :class:`QuantLib.BlackScholesCalculator`.

    Parameters
    ----------
    inp : BlackScholes.Inputs
        Standard Black-Scholes inputs *(S₀, K, T, r, q, σ)* where
        ``d`` is the dividend (repo) yield *q*.

    Returns
    -------
    BlackScholes.HostPricingResults
        * **call_price**, **put_price** – analytic option premiums.  
        * **call_price_intrinsic**, **put_price_intrinsic** – discounted
          intrinsic values **based on the forward expectation**
          :math:`E_Q[S_T]=S_0e^{(r-q)T}`:  
          :math:`e^{-rT}\\max(E_Q[S_T]-K,0)` and the analogous put.  
        * **underlying** – :math:`E_Q[S_T]`.  
        * **call_convexity**, **put_convexity** – returned as ``0.0`` (unused).

    Notes
    -----
    * No term structures or pricing engines are built; we use the closed-form
      calculator directly.
    * `std_dev = σ√T`, `discount = e^{-rT}`, `growth = e^{-qT}` are the inputs
      expected by :class:`~QuantLib.BlackScholesCalculator`.
    """
    # Pre-compute closed-form factors ⟶ inputs to BlackScholesCalculator
    std_dev: Final[float] = inp.v * math.sqrt(inp.T)            # σ√T
    discount: Final[float] = math.exp(-inp.r * inp.T)           # e^{-rT}
    growth: Final[float] = math.exp(-inp.d * inp.T)             # e^{-qT}

    # Create calculators for call and put
    call_calc = ql.BlackScholesCalculator(
        ql.Option.Call, inp.K, inp.X0, growth, std_dev, discount
    )
    put_calc = ql.BlackScholesCalculator(
        ql.Option.Put, inp.K, inp.X0, growth, std_dev, discount
    )

    # Forward expectation of the underlying at expiry
    expected_st: float = inp.X0 * math.exp((inp.r - inp.d) * inp.T)

    # Discounted intrinsic values based on expected_st
    call_intrinsic: float = discount * max(expected_st - inp.K, 0.0)
    put_intrinsic: float = discount * max(inp.K - expected_st, 0.0)

    return BlackScholes.HostPricingResults(
        call_price_intrinsic=call_intrinsic,
        put_price_intrinsic=put_intrinsic,
        underlying=expected_st,
        put_convexity=0.0,
        call_convexity=0.0,
        call_price=call_calc.value(),
        put_price=put_calc.value(),
    )

In [ ]:
%%time
ql_prices=[ql_black_scholes_pricer(input) for input in samples]

In [ ]:
prices[0]

In [ ]:
ql_prices[0]